<a href="https://colab.research.google.com/github/BenjaminDKLuong/Colab_Notes/blob/master/Detect_Mean_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DECTECT MEAN TWEETS**

In [13]:
!git clone https://github.com/PacktPublishing/Real-World-Python-Deep-Learning-Projects.git

Cloning into 'Real-World-Python-Deep-Learning-Projects'...
remote: Enumerating objects: 13503, done.
remote: Total 13503 (delta 0), reused 0 (delta 0), pack-reused 13503
Receiving objects: 100% (13503/13503), 178.97 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (85/85), done.
Checking out files: 100% (13472/13472), done.


In [0]:
# change working directory to new location
import os
os.chdir("/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source")


In [15]:
# check current working directory
%pwd

'/content/Real-World-Python-Deep-Learning-Projects/Section 3 Code/source'

In [16]:
!ls

conf.py  mean_or_not.py  prep.py      stopwords.py
data	 models		 __pycache__  train.py


In [0]:
import pandas as pd
df = pd.read_csv('/content/sample_data/mnist_test.csv')

In [0]:
df.to_csv('mnist_copy.csv')

In [0]:
from google.colab import files
files.download('mnist_copy.csv')

In [7]:
from urllib.request import urlretrieve
url = 'https://github.com/PacktPublishing/Real-World-Python-Deep-Learning-Projects/blob/master/Section%203%20Code/source/data/txt_sentoken/neg/cv024_7033.txt'
filename='test.txt'
urlretrieve(url,filename)

('test.txt', <http.client.HTTPMessage at 0x7f2079f02400>)

In [10]:
!ls

example.csv  mnist_copy.csv  sample_data  test.txt


In [11]:
%pwd

'/content'

In [12]:
import tensorflow as tf
tf.test.gpu_device_name()


''